### Organiza o resultado do algoritmo apriori
Após executar o algoritmo apriori, o resultado sofre algumas modificações facilitando sua visualização.<br>
Remoção de parte da string desnecessária no arquivo resultante do algoritmo apriori.

# Bibliotecas

In [1]:
import pandas as pd
from decimal import Decimal

# Carregamento dos dados

In [2]:
# Nome do arquivo com o resultado apriori
nomeArq = 'AprioriMlxtend_ES_datatran2019_Apriori'

In [3]:
data = pd.read_csv(nomeArq +'.csv', encoding='ISO-8859-1', sep=';')

In [4]:
data.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
0,"frozenset({'feridos=1', 'br=101.0', 'pessoas=1'})","frozenset({'ignorados=0', 'ilesos=0', 'classif...",0.130542,0.286066,0.130542,1.0,3.495695,0.093198,inf,3,3
1,"frozenset({'pessoas=1', 'br=101.0', 'classific...","frozenset({'ignorados=0', 'feridos=1', 'ilesos...",0.130542,0.183673,0.130542,1.0,5.444444,0.106565,inf,3,3
2,"frozenset({'feridos=1', 'br=101.0', 'pessoas=1'})","frozenset({'ignorados=0', 'mortos=0', 'classif...",0.130542,0.732231,0.130542,1.0,1.365690,0.034955,inf,3,3
3,"frozenset({'pessoas=1', 'br=101.0', 'classific...","frozenset({'ignorados=0', 'feridos=1', 'mortos...",0.130542,0.525686,0.130542,1.0,1.902276,0.061918,inf,3,3
4,"frozenset({'feridos=1', 'br=101.0', 'pessoas=1'})","frozenset({'ignorados=0', 'veiculos=1', 'class...",0.130542,0.294511,0.130542,1.0,3.395460,0.092096,inf,3,3


In [5]:
data.shape

(2837, 11)

In [6]:
data.columns

Index(['antecedents', 'consequents', 'antecedent support',
       'consequent support', 'support', 'confidence', 'lift', 'leverage',
       'conviction', 'antecedent_len', 'consequent_len'],
      dtype='object')

# Organização dos dados

## Remoção de parte da string desnecessária (antecedents, consequents)

In [7]:
tamLinha = data.shape[0]

In [8]:
Ant_Suc = []
Ant_Suc.append('antecedents')
Ant_Suc.append('consequents')
# Ant_Suc

In [9]:
for c in Ant_Suc:
    for i in range(tamLinha):
        # Remoção do 'frozenset(' e ')'
        data.at[i, c] = data.at[i, c].replace("frozenset(", "")
        data.at[i, c] = data.at[i, c].replace(")", "")
        
        # Remoção do 'nan'
        data.at[i, c] = data.at[i, c].replace("'nan'", "")
        data.at[i, c] = data.at[i, c].replace("{}", "")
        
        # Remoção de alguns casos depois da remoção do 'nan'
        data.at[i, c] = data.at[i, c].replace(" , ", " ")
        data.at[i, c] = data.at[i, c].replace(",}", "}")
        data.at[i, c] = data.at[i, c].replace(", }", "}")
        data.at[i, c] = data.at[i, c].replace("{, ", "{")

## Remoção das linhas vazias

In [10]:
data.shape

(2837, 11)

In [11]:
# Remoção de linhas com antecedentes ou consequenes nulos
df_remove = data.loc[((data['antecedents'] == '') | (data['consequents'] == ''))]
data = data.drop(df_remove.index)

In [12]:
data.shape

(2837, 11)

## Removendo Linhas duplicadas

In [13]:
data.shape

(2837, 11)

In [14]:
data.drop_duplicates(inplace=True) # Verifica a existência de linhas duplicadas

In [15]:
# Ordena de acordo com a coluna support de forma decrescente
data.sort_values(by=['confidence', 'support'], inplace=True, ascending=False)

# Remoção de regras similares

In [16]:
# data.sort_values(by=['support'], ascending=False, inplace=True)

In [17]:
# Gambiarra para atualizar o arquivo com os dados ordenados, de acordo com a célula anterior
data.to_csv(nomeArq + '_Backup.csv', encoding= 'ISO-8859-1', sep= ';', index=None)
data = pd.read_csv(nomeArq +'_Backup.csv', encoding='ISO-8859-1', sep=';')

In [18]:
def asterix(num):
    a = '*'
    x =''
    for i in range(num):
        x = x + a    
    return x

In [19]:
%%time
tamLinha = data.shape[0]
flag = 1
listRemove = []
for i in range(0, tamLinha):
    flag = 1
    for j in range(i+1, tamLinha):
        if ( (data.at[i, 'antecedents'] == data.at[j, 'consequents']) and (data.at[i, 'consequents'] == data.at[j, 'antecedents'])): # testar equals
                count = i+2 
                data.at[j, 'antecedents'] = str(count) + (asterix(flag))
                data.at[j, 'consequents'] = str(count) + (asterix(flag))
#                 data.at[j, 'antecedents'] = ''
#                 data.at[j, 'consequents'] = ''
                listRemove.append(j) # Adiciona em uma lista as linhas que serão removidas logo em seguida
                flag = flag + 1

Wall time: 1min 25s


In [20]:
# A lista possui o número das linhas que possuem dados repetidos e serão removidas
for r in listRemove:
#     print("Linha:", r)
    data.drop(r, inplace=True)

print("Nº de linhas removidas: ", len(listRemove))

Nº de linhas removidas:  13


In [21]:
data.shape

(2824, 11)

# Limitar casas decimais

In [22]:
decimals = 8

listCol = ['antecedent support', 'consequent support', 'support', 'confidence', 'lift', 'leverage', 'conviction']
for i in listCol:
    data[i] = data[i].apply(lambda x: round(x, decimals))


# Incluir uma nova coluna com a identificação de cada linha

In [23]:
#Célula utilizada para atualizar a linha do dataframe. Iniciando a linha em 0.
# Salvando em um arquivo .csv
data.to_csv(nomeArq + '_Backup.csv', encoding= 'ISO-8859-1', sep= ';', index=False)

# Gambiarra para atualizar o arquivo com os dados ordenados, de acordo com a célula anterior
data = pd.read_csv(nomeArq + '_Backup.csv', encoding='ISO-8859-1', sep=';')

In [24]:
# Cria uma nova coluna que servirá para identificar cada regra de associação com um número inteiro.
idx = 0
new_col = 0
data.insert(loc=idx, column='n_row', value=new_col)

row =1
for i in range(data.shape[0]):
    data.at[i, 'n_row'] = int(row)
    row = row + 1

# Substituição de '.' para ','

In [25]:
# Fazer essa substituição nas demais colunas

In [26]:
# data['lift'] = data['lift'].astype(str)
# data['lift'] = data['lift'].str.replace('.', ',')

In [27]:
listCol = ['antecedent support', 'consequent support', 'support', 'confidence', 'lift', 'leverage', 'conviction']
for i in listCol:
    data[i] = data[i].astype(str)
    data[i] = data[i].str.replace('.', ',')

# Resultado final

In [28]:
data.head(10)

,n_row,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
0,1,"{'pessoas=2', 'feridos=1', 'ilesos=1'}","{'ignorados=0', 'mortos=0', 'classificacao_aci...","0,26706545","0,73223082","0,26706545","1,0","1,36568957","0,07151189",inf,3,3
1,2,"{'pessoas=2', 'ilesos=1', 'classificacao_acide...","{'ignorados=0', 'feridos=1', 'mortos=0'}","0,26706545","0,52568614","0,26706545","1,0","1,90227577","0,12667284",inf,3,3
2,3,"{'pessoas=2', 'feridos=1', 'ilesos=1', 'veicul...","{'ignorados=0', 'mortos=0', 'classificacao_aci...","0,22765658","0,73223082","0,22765658","1,0","1,36568957","0,06095941",inf,4,3
3,4,"{'pessoas=2', 'ilesos=1', 'veiculos=2', 'class...","{'ignorados=0', 'feridos=1', 'mortos=0'}","0,22765658","0,52568614","0,22765658","1,0","1,90227577","0,10798067",inf,4,3
4,5,"{'feridos=1', 'pessoas=2', 'br=101.0', 'ilesos...","{'ignorados=0', 'mortos=0', 'classificacao_aci...","0,19528501","0,73223082","0,19528501","1,0","1,36568957","0,05229131",inf,4,3
5,6,"{'pessoas=2', 'br=101.0', 'ilesos=1', 'classif...","{'ignorados=0', 'feridos=1', 'mortos=0'}","0,19528501","0,52568614","0,19528501","1,0","1,90227577","0,09262639",inf,4,3
6,7,"{'feridos=1', 'mortos=0', 'pessoas=1'}","{'ignorados=0', 'ilesos=0', 'classificacao_aci...","0,18121042","0,28606615","0,18121042","1,0","3,49569496","0,12937225",inf,3,3
7,8,"{'pessoas=1', 'feridos=1', 'classificacao_acid...","{'ignorados=0', 'mortos=0', 'ilesos=0'}","0,18121042","0,28606615","0,18121042","1,0","3,49569496","0,12937225",inf,3,3
8,9,"{'ignorados=0', 'feridos=1', 'pessoas=1'}","{'mortos=0', 'ilesos=0', 'classificacao_aciden...","0,18121042","0,34658691","0,18121042","1,0","2,88527919","0,11840526",inf,3,3
9,10,"{'feridos=1', 'ilesos=0', 'pessoas=1'}","{'ignorados=0', 'mortos=0', 'classificacao_aci...","0,18121042","0,73223082","0,18121042","1,0","1,36568957","0,04852256",inf,3,3


# Salvando o arquivo resultante

In [29]:
data.to_csv(nomeArq + '_Mod.csv', encoding= 'ISO-8859-1', sep= ';', index=None)

In [30]:
print("ACABOU!")

ACABOU!
